In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install --upgrade pip

In [ ]:
pip install tensorflow-gpu==2.2

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
cd /content/models/research

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
pip install tf_slim

In [ ]:
pip install pandas

In [ ]:
!unzip "/content/drive/My Drive/yolov3/images.zip" -d /content/models/research/object_detection

In [ ]:
!unzip "/content/drive/My Drive/yolov3/data.zip" -d /content/models/research/object_detection

In [ ]:
cd /content/models/research/object_detection

In [ ]:
!mkdir training

In [ ]:
cd training/

In [ ]:
!touch object-detection.pbtxt

In [ ]:
cd /content/models/research/object_detection

In [ ]:
pip install pycocotools

In [ ]:
pip install scipy

In [ ]:
pip install dataclasses

In [ ]:
pip install pyyaml

In [ ]:
pip install tf_slim

In [ ]:
cd /content/models/research/

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cp /content/drive/MyDrive/yolov3/setup.py /content/models/research

In [ ]:
!export PYTHONPATH=$PYTHONPATH:'pwd':'pwd'/slim

In [ ]:
!python setup.py build

In [ ]:
!python setup.py install

In [ ]:
cd /content/models/research/object_detection

In [ ]:
pip install tf-models-official

In [ ]:
cd /content/models/research/slim

In [ ]:
!python setup.py build

In [ ]:
!python setup.py install

In [ ]:
!export PYTHONPATH="$PYTHONPATH:/home/models:/content/research/slim"

In [ ]:
cd /content/models/research/object_detection

In [ ]:
#!tar -xvzf /content/models/research/object_detection/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

In [ ]:
pip install pycocotools

In [ ]:
pip install scipy

In [ ]:
pip install lvis

In [ ]:
cd /content/models/research/object_detection 

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar -xvzf /content/models/research/object_detection/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

In [ ]:
!python model_main_tf2.py --pipeline_config_path=/content/models/research/object_detection/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config --model_dir=/content/models/research/object_detection/training/training_check_point --num_train_steps=3500 --sample_1_of_n_eval_examples=1 --num_eval_steps=1


In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/models/research/object_detection/training/pipeline_v2.config --trained_checkpoint_dir /content/models/research/object_detection/training/training_check_point --output_directory /content/models/research/object_detection/exported-mobile-model

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path /content/models/research/object_detection/training/pipeline_v2.config --trained_checkpoint_prefix /content/models/research/object_detection/training/training_check_point/ckpt-3.data-00000-of-00001 --output_directory /content/models/research/object_detection/exported_model_directory

In [ ]:
!python export_tflite_graph_tf2.py --pipeline_config_path /content/models/research/object_detection/training/pipeline_resnet.config --trained_checkpoint_dir /content/models/research/object_detection/training/training_check_point --output_directory /content/models/research/object_detection/tflite_new

In [ ]:
!cp -R /content/models/research/object_detection/tflite_new /content/drive/MyDrive/yolov3

In [ ]:
!cp -R /content/models/research/object_detection/exported-mobile-model_512 /content/drive/MyDrive/yolov3

In [ ]:
pip install tensorflow==2.1.0

In [ ]:
pip install tensorflow --upgrade

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
#testing the result
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
pip uninstall tensorflow-object-detection-api

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


In [ ]:
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:

def load_model(model_name):
  #base_url = 'http://download.tensorflow.org/models/object_detection/'
  #model_file = model_name + '.tar.gz'
  #model_dir = tf.keras.utils.get_file(
  #  fname=model_name, 
  #  origin=base_url + model_file,
  #  untar=True)

  model_dir = "/content/models/research/object_detection/exported-mobile-model/saved_model"
  
  model_dir = '/content/models/research/object_detection/exported-mobile-model/saved_model'
  model = tf.saved_model.load(str(model_dir))

  return model

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/models/research/object_detection/training/object-detection.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
import os
import pathlib

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/models/research/object_detection/images/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

In [ ]:
detection_model.signatures['serving_default'].output_shapes

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  #print("num_detection ",output_dict)
  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

In [ ]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model(model_name)

In [ ]:
masking_model.output_shapes

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(masking_model, image_path)

In [ ]:
#pip install tensorflow-object-detection-api
#if we got import error tensorflow object detection

In [ ]:
# converting the tflite model
import tensorflow as tf
model = tf.saved_model.load("/content/models/research/object_detection/tflite_new/saved_model") 
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 1024, 1024, 3]) 
tf.saved_model.save(model, "saved_model_updated", signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]) 
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='/content/models/research/object_detection/tflite_new/saved_model', signature_keys=['serving_default']) 
converter.optimizations = [tf.lite.Optimize.DEFAULT] 
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS] 
tflite_model = converter.convert() 
open("/content/ssd_mobile_1_new.tflite", "wb").write(tflite_model)


In [ ]:
!touch labelmap.txt

In [ ]:
!ls

In [ ]:
pip install tflite-support

In [ ]:
#pip install tflite-support
import os
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb


# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"
model_meta.description = ("Identify Unesco Monuments Route"
                          "image from a set of 18 categories")
model_meta.version = "v1"
model_meta.author = "TensorFlow"
model_meta.license = ("Apache License. Version 2.0 "
                      "http://www.apache.org/licenses/LICENSE-2.0.")


# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()


input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(300, 300))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats



# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the 18 labels respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename('/content/models/research/object_detection/training/label_map.txt')
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]


# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = 4*[output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()


# metadata and the label file are written into the TFLite file
populator = _metadata.MetadataPopulator.with_model_file('/content/ssd_mobile_1_new.tflite')
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files(['/content/models/research/object_detection/training/label_map.txt'])
populator.populate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cp -R /content/drive/MyDrive/yolov3/exported-mobile-model /content/

In [ ]:
cp -R /content/drive/MyDrive/yolov3/exported_model_directory /content/